In [1]:
from sqlalchemy import create_engine, Column, String, Integer, func, event, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.orm import sessionmaker
from geoalchemy2 import Geometry 
from tqdm import tqdm
from shapely.wkt import dumps

import orjson


In [2]:
# Create a base class for our declarative mapping
Base = declarative_base()

# Define your SQLAlchemy model
class GeometryModel(Base):
    __tablename__ = 'geometries'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    geom = Column(Geometry('POLYGON'))

/tmp/ipykernel_84458/2237727833.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
from sqlalchemy_utils import database_exists, create_database
engine = create_engine('postgresql://postgres@localhost:5333/test')#,echo=True)


In [4]:
# Initialize Spatialite extension
@event.listens_for(engine, "connect")
def connect(dbapi_connection, connection_record):
    with dbapi_connection.cursor() as cursor:
        cursor.execute('CREATE EXTENSION IF NOT EXISTS postgis;')

In [5]:
# Create the table
Base.metadata.create_all(engine)

# Start a session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
%%time
results = session.query(GeometryModel).all()

In [41]:
len(results)

1063260

In [42]:
%%time
# Detach all results from the session
# expunge_all() removes all objects from the session, making them detached. Detached objects are no longer tracked by the session, so changes made to them will not be automatically persisted to the database.
session.expunge_all()

CPU times: user 287 ms, sys: 3.03 ms, total: 290 ms
Wall time: 288 ms


In [43]:
%%time
## -- Convert from EWKB to Shapely
from shapely.geometry import shape
from shapely import wkb
for r in tqdm(results):
    r.geom = wkb.loads(str(r.geom)) #note that this is different than the spatialite version

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1063260/1063260 [02:37<00:00, 6754.20it/s]

CPU times: user 2min 37s, sys: 654 ms, total: 2min 37s
Wall time: 2min 37s


In [10]:
%%time
from geoalchemy2.functions import ST_AsGeoJSON
results = session.query(ST_AsGeoJSON(GeometryModel.geom)).all()

CPU times: user 8.19 s, sys: 1.98 s, total: 10.2 s
Wall time: 30.8 s


In [11]:
results[0]

('{"type":"Polygon","coordinates":[[[53109,13464],[53109,13465],[53109,13466],[53109,13467],[53109,13468],[53108,13469],[53107,13470],[53106,13471],[53 ... (10560 characters truncated) ... 13464],[53118,13464],[53117,13464],[53116,13464],[53115,13464],[53114,13464],[53113,13464],[53112,13464],[53111,13464],[53110,13464],[53109,13464]]]}',)

In [12]:
# %%time
# from geoalchemy2.functions import ST_AsEWKT
# results = session.query(ST_AsEWKT(GeometryModel.geom)).all()

In [13]:
%%time
from geoalchemy2.functions import ST_Centroid
results = session.query(ST_Centroid(GeometryModel.geom)).all()

CPU times: user 7.64 s, sys: 312 ms, total: 7.95 s
Wall time: 13.3 s


In [14]:
len(results)

1063260

In [15]:
results[0]

(<WKBElement at 0x7f0c30c9a3e0; 0101000020e6100000e6daad49a7f7e9405afb777be186ca40>,)

In [16]:
%%time
from geoalchemy2.functions import ST_Centroid
results = session.query(ST_AsGeoJSON(ST_Centroid(GeometryModel.geom))).all()

CPU times: user 2.55 s, sys: 226 ms, total: 2.78 s
Wall time: 7.95 s


In [17]:
# %%time
results = session.query(GeometryModel.geom).limit(10).all()